In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
# activate R magic
%load_ext rpy2.ipython

/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:14: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import Index as PandasIndex
/usr/local/lib/python3.6/dist-packages/rpy2/robjects/pandas2ri.py:34: UserWarning: pandas >= 1.0 is not supported.
  warnings.warn('pandas >= 1.0 is not supported.')


In [3]:
%%R
install.packages("lme4")
install.packages("feather")
library("lme4")
library(feather)
library("dplyr")

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘minqa’, ‘nloptr’, ‘statmod’, ‘RcppEigen’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/minqa_1.2.4.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 53548 bytes (52 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[wr

In [0]:
%%R
review_dta <- read_feather("/content/drive/My Drive/Dissertation/airbnb/cultural distance/review.feather")

In [0]:
%%R
pvars <- c("power_distance","individualism","masculinity","uncertainty_avoidance"
            ,"LT_orientation","indulgence"
            ,"dst","gc_dst" ,"age"
            ,"host_listings_count","number_of_reviews","price","bathrooms","bedrooms"
            ,"review_scores_location"
            ,"review_scores_rating"
            ,"review_scores_accuracy")
datsc <- review_dta
datsc[pvars] <- lapply(datsc[pvars],scale)
datsc <- cbind(datsc, select(review_dta, vader_sent, gender, host_is_superhost))

In [0]:
%%R
m1 <- glmer(vader_sent ~ power_distance + individualism + masculinity + uncertainty_avoidance
            + LT_orientation + indulgence
            + dst + gc_dst + age + gender
            #+ host_is_superhost + host_listings_count + number_of_reviews + price + bathrooms + bedrooms
            #+ review_scores_location
            + review_scores_rating + individualism:review_scores_rating
            #+ review_scores_accuracy + individualism:review_scores_accuracy
            + LT_orientation:gc_dst
            + power_distance:age
            #+ power_distance:price
            + (1 | listing_id)
            #+ (1 | country)
            , data = datsc, family = binomial, nAGQ = 1)

In [17]:
%%R
summary(m1)

Generalized linear mixed model fit by maximum likelihood (Laplace
  Approximation) [glmerMod]
 Family: binomial  ( logit )
Formula: 
vader_sent ~ power_distance + individualism + masculinity + uncertainty_avoidance +  
    LT_orientation + indulgence + dst + gc_dst + age + gender +  
    host_is_superhost + host_listings_count + number_of_reviews +  
    price + bathrooms + bedrooms + review_scores_location + review_scores_rating +  
    individualism:review_scores_rating + review_scores_accuracy +  
    individualism:review_scores_accuracy + LT_orientation:gc_dst +  
    power_distance:age + power_distance:price + (1 | listing_id) +  
    (1 | country)
   Data: datsc

     AIC      BIC   logLik deviance df.resid 
 51858.1  52142.5 -25902.1  51804.1   277253 

Scaled residuals: 
     Min       1Q   Median       3Q      Max 
-16.8496   0.0800   0.1049   0.1555  12.1779 

Random effects:
 Groups     Name        Variance Std.Dev.
 listing_id (Intercept) 0.07176  0.2679  
 country    (Inte

R[write to console]: 
Correlation matrix not shown by default, as p = 25 > 12.
Use print(object, correlation=TRUE)  or
    vcov(object)        if you need it




convergence code: 0
Model failed to converge with max|grad| = 0.0437763 (tol = 0.002, component 1)



In [0]:
%%R
save(m1, file = "/content/drive/My Drive/Dissertation/airbnb/cultural distance/m.rda")

In [0]:
load("m1.rda")